In [98]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import re
import pandas as pd
import json

In [99]:
#Идея привести все ЗП к одному виду мне понравилась, попробовал тоже реализовать это.
cur_req = requests.get("https://www.cbr-xml-daily.ru/daily_json.js")    
cur = json.loads(cur_req.text)    

In [117]:
job_info = []
n = 0
max_list = 30
for x in range(max_list):
    resphh = requests.get(f'https://hh.ru/search/vacancy?area=1&fromSearchLine=true&text=Python&from=suggest_post&page={x}', headers = {'User-agent':'Mozilla/5.0'}).text
    soup = bs(resphh, 'html.parser')
    cards = soup.find_all(name='div', attrs={"class" : "vacancy-serp-item"} )
    for i in cards:
        name_job = i.find('a', attrs = {'data-qa': "vacancy-serp__vacancy-title"}).text
        
        link_job = i.find('a', attrs = {'data-qa': "vacancy-serp__vacancy-title"}).get('href')
        
        cost = i.find('span', attrs = {'data-qa' : "vacancy-serp__vacancy-compensation"})
        min_cost = np.nan
        max_cost = np.nan
        if cost:
            cost = cost.text 
            cost = cost.replace("\u202f", "").replace(" ", "")
            if re.findall('^от[0-9]+', cost):
                min_cost = int(re.findall('[0-9]+', cost)[0])
            elif re.findall('^до[0-9]+', cost):
                max_cost = int(re.findall('[0-9]+', cost)[0])
            else:
                min_cost = int(re.findall('[0-9]+', cost)[0])
                max_cost = int(re.findall('[0-9]+', cost)[1])
            if not re.findall('руб.', cost):
                currency = re.findall('[A-Z]{2,5}', cost)[0]
                if not np.isnan(min_cost):
                    min_cost = int(min_cost*cur["Valute"][currency]["Value"])//1000*1000
                if not np.isnan(max_cost):
                    max_cost = int(max_cost*cur["Valute"][currency]["Value"])//1000*1000
        
        job_from = "hh.ru"
        job_info.append([])
        job_info[n].append(name_job)
        job_info[n].append(link_job)
        job_info[n].append(min_cost)
        job_info[n].append(max_cost)
        job_info[n].append(job_from)
        
        n+=1    



In [118]:
columns = ["Name job", "Link", "Min_salary", "Max_salary", "Where"]

In [119]:
df = pd.DataFrame(job_info, columns=columns)

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Name job    600 non-null    object 
 1   Link        600 non-null    object 
 2   Min_salary  187 non-null    float64
 3   Max_salary  152 non-null    float64
 4   Where       600 non-null    object 
dtypes: float64(2), object(3)
memory usage: 23.6+ KB
